In [0]:
%pip install langchain
dbutils.library.restartPython()

  Obtaining dependency information for langchain from https://files.pythonhosted.org/packages/51/3f/462c134228fbb4f65be0a9db6a651e2f1d7226d003a712f1bac455a141b7/langchain-0.3.1-py3-none-any.whl.metadata
  Obtaining dependency information for SQLAlchemy<3,>=1.4 from https://files.pythonhosted.org/packages/8c/d6/97bdc8d714fb21762f2092511f380f18cdb2d985d516071fa925bb433a90/SQLAlchemy-2.0.35-cp311-cp311-manylinux_2_17_x86_64.manylinux2014_x86_64.whl.metadata
  Obtaining dependency information for aiohttp<4.0.0,>=3.8.3 from https://files.pythonhosted.org/packages/fd/80/4f1c4b5459a27437a8f18f91d6000fdc45b677aee879129deaadc94c1a23/aiohttp-3.10.6-cp311-cp311-manylinux_2_17_x86_64.manylinux2014_x86_64.whl.metadata
  Obtaining dependency information for langchain-core<0.4.0,>=0.3.6 from https://files.pythonhosted.org/packages/3e/0b/f189d87ccc7a5f388649bbf7e513ad774b14d66a4bb96278cb211347a17c/langchain_core-0.3.6-py3-none-any.whl.metadata
  Obtaining dependency information for langchain-text-spli

In [0]:
import os
from langchain.text_splitter import RecursiveCharacterTextSplitter

# Read the text file
df = spark.read.text("dbfs:/Volumes/rea-onyx-uc-catalog/rea_onyx_uc_schema/rea_test_volume/physicalasset.csv")

# Collect all the text into a single string
text_column = " ".join([row.value for row in df.collect()])

length_function = len

splitter = RecursiveCharacterTextSplitter(
    separators=["\n\n", "\n", " ", ""],
    chunk_size=1000,
    chunk_overlap=200,
    length_function=length_function,
)
chunks = splitter.split_text(text_column)

In [0]:
from pyspark.sql.functions import pandas_udf
from pyspark.sql.types import ArrayType, StringType
import pandas as pd

@pandas_udf("array<string>")
def get_chunks(dummy):
    return pd.Series([chunks])

# Register the UDF
spark.udf.register("get_chunks_udf", get_chunks)

In [0]:
%sql
insert into `rea-onyx-uc-catalog`.rea_onyx_uc_schema.docs_text (text)
select explode(get_chunks_udf('dummy')) as text;

num_affected_rows,num_inserted_rows
104569,104569


In [0]:
df.createOrReplaceTempView("temp_table")  # Create a temporary table from the DataFrame

# Insert only the rows that do not exist in the target table using a left anti join
spark.sql("""
    INSERT INTO `rea-onyx-uc-catalog`.rea_onyx_uc_schema.docs_track
    SELECT temp_table.*
    FROM temp_table
    LEFT ANTI JOIN `rea-onyx-uc-catalog`.rea_onyx_uc_schema.docs_track
    ON temp_table.value = `rea-onyx-uc-catalog`.rea_onyx_uc_schema.docs_track.file_name
""")

DataFrame[num_affected_rows: bigint, num_inserted_rows: bigint]